In [6]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"  # Avoid some crashes on M1/M2


In [7]:
%pwd

'/Users/pranjal/Desktop/data_science_projects/chicken-disease-classfication'

In [8]:
os.chdir("../")

In [9]:
%pwd

'/Users/pranjal/Desktop/data_science_projects'

In [21]:
import os
from pathlib import Path

# Force notebook to use project root as working directory
project_root = Path("/Users/pranjal/Desktop/data_science_projects/chicken-disease-classfication")
os.chdir(project_root)

print("Working directory set to:", Path.cwd())
print("Config path exists:", (project_root / "config/config.yaml").exists())


Working directory set to: /Users/pranjal/Desktop/data_science_projects/chicken-disease-classfication
Config path exists: True


In [23]:
import tensorflow as tf
from keras import models as standalone_keras_models
from tensorflow import keras as tf_keras_models



In [25]:
_original_cce = tf.keras.losses.CategoricalCrossentropy

def patched_categorical_crossentropy(*args, **kwargs):
    # Remove the invalid 'fn' argument that breaks loading
    kwargs.pop("fn", None)
    return _original_cce(*args, **kwargs)

# Apply patch
tf.keras.losses.CategoricalCrossentropy = patched_categorical_crossentropy

# Patch load_model for both Keras backends
_original_load_model_tf = tf_keras_models.models.load_model
_original_load_model_keras = standalone_keras_models.load_model

def safe_load_model(path, *args, **kwargs):
    return _original_load_model_tf(
        path,
        custom_objects={"CategoricalCrossentropy": patched_categorical_crossentropy},
        *args, **kwargs
    )

tf_keras_models.models.load_model = safe_load_model
standalone_keras_models.load_model = safe_load_model

In [26]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list



@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [27]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [28]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config
    




    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chicken_disease_images")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [29]:
import time

In [35]:
import os
import time
import tensorflow as tf
from cnnClassifier.entity.config_entity import PrepareCallbacksConfig

class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            str(self.config.tensorboard_root_log_dir),  # Convert Path → str
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.config.checkpoint_model_filepath),  # Convert Path → str
            save_best_only=True
        )

    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]


In [36]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [37]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    
    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [38]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e

[2025-07-25 14:21:52,362: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-07-25 14:21:52,365: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-25 14:21:52,366: INFO: common: created directory at: artifacts]
[2025-07-25 14:21:52,367: INFO: common: created directory at: artifacts/prepare_callbacks]
[2025-07-25 14:21:52,367: INFO: common: created directory at: artifacts/prepare_callbacks/tensorboard_log_dir]
[2025-07-25 14:21:52,372: INFO: common: created directory at: artifacts/training]


2025-07-25 14:21:52.468570: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2025-07-25 14:21:52.468744: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-07-25 14:21:52.469116: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-07-25 14:21:52.469858: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-07-25 14:21:52.470254: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


[2025-07-25 14:21:52,965: WARNING: optimizer: At this time, the v2.11+ optimizer `tf.keras.optimizers.SGD` runs slowly on M1/M2 Macs, please use the legacy Keras optimizer instead, located at `tf.keras.optimizers.legacy.SGD`.]
Found 100 images belonging to 2 classes.
Found 404 images belonging to 2 classes.
Epoch 1/10


2025-07-25 14:21:53.729739: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2025-07-25 14:21:53.755795: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node SGD/AssignVariableOp.


12/12 [==============================] - 8s 629ms/step - loss: 0.7913 - accuracy: 0.6317 - val_loss: 0.8628 - val_accuracy: 0.4896
Epoch 2/10


/Users/pranjal/Desktop/data_science_projects/chicken-disease-classfication/.venv/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


12/12 [==============================] - 7s 579ms/step - loss: 0.8952 - accuracy: 0.5242 - val_loss: 1.2470 - val_accuracy: 0.4792
Epoch 3/10
12/12 [==============================] - 7s 579ms/step - loss: 0.6032 - accuracy: 0.6962 - val_loss: 0.7018 - val_accuracy: 0.5208
Epoch 4/10
12/12 [==============================] - 7s 582ms/step - loss: 0.5518 - accuracy: 0.6935 - val_loss: 0.3256 - val_accuracy: 0.8750
Epoch 5/10
12/12 [==============================] - 7s 592ms/step - loss: 0.3847 - accuracy: 0.8629 - val_loss: 0.2994 - val_accuracy: 0.8542
Epoch 6/10
12/12 [==============================] - 7s 572ms/step - loss: 0.2895 - accuracy: 0.8925 - val_loss: 0.3412 - val_accuracy: 0.8542
Epoch 7/10
12/12 [==============================] - 7s 576ms/step - loss: 0.3501 - accuracy: 0.8387 - val_loss: 0.2538 - val_accuracy: 0.9062
Epoch 8/10
12/12 [==============================] - 7s 589ms/step - loss: 0.2970 - accuracy: 0.8932 - val_loss: 0.2361 - val_accuracy: 0.9167
Epoch 9/10
12/12 